# 出貨文件辨識

In [90]:
import pytesseract
from PIL import Image
import re
import csv
import pandas as pd
import numpy as np
import math
import tabula
from glob import glob

import cv2
import numpy as np
import pytesseract

import aircv as ac
import pyautogui
import matplotlib.pyplot as plt

import re
import sys
import fitz
pytesseract.pytesseract.tesseract_cmd = 'C://Program Files/TesseractOCR/tesseract.exe'

In [91]:
#要辨識的PDF
import shutil
import os 
invoice_pdf_path="8000102855IN.pdf"

#重建資料夾
shutil.rmtree('./invoice_pdf_jpg_result')
os.mkdir('./invoice_pdf_jpg_result')




# PDF轉JPG

In [92]:
#PDF轉JPG
import glob
zoom_x = 4.0  # horizontal zoom
zoom_y = 4.0  # vertical zoom
mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension

all_files = glob.glob(invoice_pdf_path)

for filename in all_files:
    doc = fitz.open(filename)  # open document
    for page in doc:  # iterate through the pages
        pix = page.get_pixmap(matrix=mat)  # render page to an image
        pix.save("./invoice_pdf_jpg_result/page-%i.png" % page.number)  # store image as a PNG
        
print("總共"+str(int(page.number)+1)+"頁" )

總共1頁


# 切表格

In [93]:
#辨識出表格位置
for i in range(int(page.number)+1):
    pdfimg_path="./invoice_pdf_jpg_result/page-"+str(i)+".png"
    image = cv2.imread(pdfimg_path)

    gray_scale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    th1,img_bin = cv2.threshold(gray_scale,150,225,cv2.THRESH_BINARY)

    canvas_black = np.zeros((image.shape[0],image.shape[1],image.shape[2]), dtype="uint8")

    img_bin=~img_bin

    line_min_width = 15
    kernal_h = np.ones((1,line_min_width), np.uint8)
    kernal_v = np.ones((line_min_width,1), np.uint8)

    img_bin_h = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernal_h)
    img_bin_v = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernal_v)

    img_bin_final=img_bin_h|img_bin_v

    _, labels, stats,_ = cv2.connectedComponentsWithStats(~img_bin_final, connectivity=8, ltype=cv2.CV_32S)

    for x,y,w,h,area in stats[2:]:
        cv2.rectangle(canvas_black,(x,y),(x+w,y+h),(255,255,255),-1)
    
    cv2.imwrite("./table_result/canvas_black"+str(i)+".jpg",canvas_black)

# 依照canvas_black的圖片裁剪出表格與非表格

In [94]:

for i in range(int(page.number)+1):
    print(i)
    pdfimg_path="./invoice_pdf_jpg_result/page-"+str(i)+".png"
    image = cv2.imread(pdfimg_path)
    non_table_img=image.copy()
    original_img=image.copy()
    
    canvas_path="./table_result/canvas_black"+str(i)+".jpg"
    canvas_black = cv2.imread(canvas_path)
    
    gray = cv2.cvtColor(canvas_black, cv2.COLOR_BGR2GRAY)
    ret, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY) 
    # 1. 膨脹和腐蝕操作的核函數
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (60, 10))  # 30 9
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 8))  # 24 6

    # 2. 膨脹一次，讓輪廓突出
    dilation = cv2.dilate(binary, element2, iterations=1)
    # 3. 腐蝕一次，去掉細節，如表格線等。注意這裡去掉的是豎直的線
    erosion = cv2.erode(dilation, element1, iterations=1)
    # 4. 再次膨脹，讓輪廓明顯一些
    dilation2 = cv2.dilate(erosion, element2, iterations=3)

    # 7. 存儲中間圖片
    #cv2.imwrite("./table_result/binary"+str(i)+".png", binary)
    #cv2.imwrite("./table_result/dilation"+str(i)+".png", dilation)
    #cv2.imwrite("./table_result/erosion"+str(i)+".png", erosion)
    cv2.imwrite("./table_result/dilation2"+str(i)+".png", dilation2)

    #用二次膨脹的圖繼續處理

    # 5. 查找輪廓
    region = []
    contours, hierarchy = cv2.findContours(dilation2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # 6. 篩選那些面積小的
    for j in range(len(contours)):
        cnt = contours[j]
        # 計算該輪廓的面積
        area = cv2.contourArea(cnt)
        # 面積小的都篩選掉
        if(area < 10000):
            continue
        # 找到最小的矩形，該矩形可能有方向
        rect = cv2.minAreaRect(cnt)
        # box是四個點的坐標
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        # 計算高和寬
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
        region.append(box)

    # 7.裁剪原始影像並儲存
    table_counter=0
    
    #氣泡迴圈
    for m in range(0,len(region)):
        for n in range(m+1,len(region)):
            if region[m][1][1] > region[n][1][1]:
                region[m],region[n]=region[n],region[m]     
    for box in region:
        Xs = [i[0] for i in box]
        Ys = [i[1] for i in box]
        x1 = min(Xs)
        x2 = max(Xs)
        y1 = min(Ys)
        y2 = max(Ys)
        hight = y2 - y1
        width = x2 - x1
        crop_img = original_img[y1:y1+hight, x1:x1+width]#裁減表格範圍
        cv2.rectangle(non_table_img,(x1,y1),(x1+width,y1+hight),(255,255,255),-1)#塗抹表格範圍，凸顯非表格

        #8.存進table_result
        cv2.imwrite("./table_result/table"+str(i)+str(table_counter)+".jpg", crop_img)
        table_counter=table_counter+1

    #9.存進table_result 但是是存非表格
    cv2.imwrite("./table_result/non_table"+str(i)+".jpg", non_table_img)

0


# 辨識表格內容

In [95]:
#辨識出每一個表格的格子位置
import glob
table_path = glob.glob(r'./table_result/table*')
print(len(table_path))

all_table_text_list=[]
all_row=[]

for i in range(len(table_path)):
    image = cv2.imread(table_path[i])
    #找出格子的xywh
    gray_scale=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    th1,img_bin = cv2.threshold(gray_scale,150,225,cv2.THRESH_BINARY)
    img_bin=~img_bin
    line_min_width = 100
    kernal_h = np.ones((1,line_min_width), np.uint8)
    kernal_v = np.ones((line_min_width,1), np.uint8)
    img_bin_h = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernal_h)
    img_bin_v = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernal_v)
    img_bin_final=img_bin_h|img_bin_v
    _, labels, stats,_ = cv2.connectedComponentsWithStats(~img_bin_final, connectivity=8, ltype=cv2.CV_32S)
    
    #裁減並辨識
    count = 0
    last_x2=0
    row=1
    table_text_list=[]

    for x,y,w,h,area in stats[2:]:
        count = count +1
        if x+w-last_x2 < 0: #如果換行了row + 1
            row=row+1
        crop_img = image[y:y+h, x:x+w]
        cv2.imwrite("table_part_result/"+str(count)+".jpg", crop_img)
        text = pytesseract.image_to_string(crop_img, lang="chi_tra")
        #print(text)
        table_text_list.append(text.replace('\n\n','\n'))
        last_x2=x+w
    #print(table_text_list)
    all_row.append(row)
    all_table_text_list.append(table_text_list)
print(all_table_text_list)

1
[['MARKS &\nNO\n', 'DESCRIPTION OF GOODS\n', 'QUANTIIY\n', 'UNITPRICE\nUSD\n', 'AMMOUNT\nUSD\n', 'MADE IN\nVIETNAM\n', 'PO#8000102923\n7/8 ELASTIC BAND WHIIE\n7/8 ELASTIC BAND BLACK\nPO#8000102956\n2-1/4 ELASTIC BAND WHIIE\n2-1/2 ELASTIC BAND WHIIE\n2 ” ELLASTIC BAND BLACK\nPO#8000102855\n1-3/4 ELASTIC BAND BLACK\n2”ELASTIC BAND BLACK\n1/2 FELASTIC BAND BLACK\n', '29,952.00\n1,584.00\n720.00\n576.00\n144.00\n4,032.00\n3,024.00\n432.00\n', '0.0700000\n0.0940000\n0.1835500\n0.2141400\n0.2094000\n0.1650000\n0.2094000\n0.0710000\n', '2,096.64\n148.90\n132.16\n123.34\n30.15\n665.28\n633.23\n30.67\n', '', 'GRAND TOTAL:\n', '40464.00 Y\n', '', '3,860.37\n', 'SAY TOTAL DOLLARS : ITHREE THOUSAND EIGHT HUNDRED SIXTY AND CENTS ITHIRTY SEVEN ONLY.\n']]


In [96]:
#重組辨識出來的list變成data frame
import pandas as pd
all_df_datas=[]
for i in range(len(all_table_text_list)):
    table_text_list_copy=all_table_text_list[i]
    datas=[]
    column=int(len(table_text_list_copy)/all_row[i])
    for j in range(all_row[i]):
        datas.append(table_text_list_copy[j * column: (j+1)*column])
    df_datas=pd.DataFrame(datas)
    df_datas.columns=[list(df_datas.loc[0])]
    df_datas=df_datas.drop([0])
    all_df_datas.append(df_datas)
    
invoice_table_df=all_df_datas[0]
invoice_table_df

,MARKS &\nNO\n,DESCRIPTION OF GOODS\n,QUANTIIY\n,UNITPRICE\nUSD\n,AMMOUNT\nUSD\n
1,MADE IN\nVIETNAM\n,PO#8000102923\n7/8 ELASTIC BAND WHIIE\n7/8 ELA...,"29,952.00\n1,584.00\n720.00\n576.00\n144.00\n4...",0.0700000\n0.0940000\n0.1835500\n0.2141400\n0....,"2,096.64\n148.90\n132.16\n123.34\n30.15\n665.2..."
2,,GRAND TOTAL:\n,40464.00 Y\n,,"3,860.37\n"


In [97]:
all_df_datas[0].to_csv("all_df_datas.csv",encoding="utf_8_sig")

In [98]:
descriotion_list=invoice_table_df["DESCRIPTION OF GOODS\n"].iloc[0][0]
descriotion_list_split=descriotion_list.split('\n')
while '' in descriotion_list_split:
    descriotion_list_split.remove('')
descriotion_list_split

['PO#8000102923',
 '7/8 ELASTIC BAND WHIIE',
 '7/8 ELASTIC BAND BLACK',
 'PO#8000102956',
 '2-1/4 ELASTIC BAND WHIIE',
 '2-1/2 ELASTIC BAND WHIIE',
 '2 ” ELLASTIC BAND BLACK',
 'PO#8000102855',
 '1-3/4 ELASTIC BAND BLACK',
 '2”ELASTIC BAND BLACK',
 '1/2 FELASTIC BAND BLACK']

In [99]:
quantity_list=invoice_table_df["QUANTIIY\n"].iloc[0][0]
quantity_list_split=quantity_list.split('\n')
while '' in quantity_list_split:
    quantity_list_split.remove('')
quantity_list_split

['29,952.00',
 '1,584.00',
 '720.00',
 '576.00',
 '144.00',
 '4,032.00',
 '3,024.00',
 '432.00']

In [100]:
output_df=[]
row=0

for i in range(len(descriotion_list_split)):
    if "PO" in descriotion_list_split[i]:
        pono= descriotion_list_split[i]
    else:
        output_df.append([pono,descriotion_list_split[i],quantity_list_split[row]])
        row=row+1
output_df

[['PO#8000102923', '7/8 ELASTIC BAND WHIIE', '29,952.00'],
 ['PO#8000102923', '7/8 ELASTIC BAND BLACK', '1,584.00'],
 ['PO#8000102956', '2-1/4 ELASTIC BAND WHIIE', '720.00'],
 ['PO#8000102956', '2-1/2 ELASTIC BAND WHIIE', '576.00'],
 ['PO#8000102956', '2 ” ELLASTIC BAND BLACK', '144.00'],
 ['PO#8000102855', '1-3/4 ELASTIC BAND BLACK', '4,032.00'],
 ['PO#8000102855', '2”ELASTIC BAND BLACK', '3,024.00'],
 ['PO#8000102855', '1/2 FELASTIC BAND BLACK', '432.00']]